# Set Your SQL Data Free with Kafka

## Setup Environment

### Database and Base Table

In [1]:
USE MASTER
IF EXISTS(SELECT * FROM sys.databases WHERE name = 'GamePlayDB')
BEGIN
  ALTER DATABASE GamePlayDB
  SET SINGLE_USER
  WITH ROLLBACK IMMEDIATE;

  DROP DATABASE GamePlayDB;
END
GO

Nonqualified transactions are being rolled back. Estimated rollback completion: 0%.

Nonqualified transactions are being rolled back. Estimated rollback completion: 100%.

Total execution time: 00:00:04.094

In [2]:
CREATE DATABASE GamePlayDB;
GO

Commands completed successfully.

Total execution time: 00:00:00.389

In [3]:
USE GamePlayDB;
GO

Commands completed successfully.

Total execution time: 00:00:00.001

In [6]:
ALTER DATABASE GamePlayDB
SET TRUSTWORTHY ON;

Commands completed successfully.

Total execution time: 00:00:00.002

In [4]:
SELECT DB_NAME();

(1 row affected)

Total execution time: 00:00:00.006

(No column name) GamePlayDB

In [5]:
CREATE TABLE dbo.tb_GamePlay
(
  RowID bigint identity PRIMARY KEY,
  UserID int NOT NULL,
  GameID int NOT NULL,
  WagerAmount bigint NOT NULL,
  PayoutAmount bigint NOT NULL,
  EventTime datetime2 NOT NULL
);
GO

Commands completed successfully.

Total execution time: 00:00:00.006

### Kafka Configuration Tables

The table below contains where the Kafka cluster(s) are.

In [6]:
CREATE TABLE dbo.tb_KafkaCluster
(
  ClusterID int,
  BootstrapServers nvarchar(4000) NOT NULL,
  [Description] nvarchar(4000) NOT NULL,
  CONSTRAINT [pk_KafkaCluster] PRIMARY KEY
  (ClusterID)
);
GO

Commands completed successfully.

Total execution time: 00:00:00.009

Table for matching events with topics

In [7]:
CREATE TABLE dbo.tb_KafkaEventSubscriber
(
  SubscriberID int identity,
  EventID int NOT NULL,
  ClusterID int NOT NULL,
  Topic nvarchar(256) NOT NULL,
  CONSTRAINT [pk_KafkaEventSubscriber] PRIMARY KEY
  (EventID, ClusterID, Topic),
  CONSTRAINT [fk_KafkaEventSubscriber_ClusterID] FOREIGN KEY (ClusterID)
  REFERENCES dbo.tb_KafkaCluster(ClusterID)
);
GO

Commands completed successfully.

Total execution time: 00:00:00.009

### Ingest Data in Config Tables

In [8]:
INSERT INTO dbo.tb_KafkaCluster(ClusterID, BootstrapServers, [Description])
VALUES(1, 'localhost:9092', 'First cluster');

(1 row affected)

Total execution time: 00:00:00.005

In [9]:
INSERT INTO dbo.tb_KafkaEventSubscriber(EventID, ClusterID, Topic)
VALUES (1, 1, 'testTopic'),
       (1500, 1, 'wagers');

(2 rows affected)

Total execution time: 00:00:00.011

Chack it worked

In [10]:
SELECT * FROM dbo.tb_KafkaCluster;

(1 row affected)

Total execution time: 00:00:00.021

ClusterID BootstrapServers Description 1 localhost:9092 First cluster

In [14]:
SELECT * FROM dbo.tb_KafkaEventSubscriber;

(2 rows affected)

Total execution time: 00:00:00.013

SubscriberID EventID ClusterID Topic 1 1 1 testTopic 2 1500 1 wagers